In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [9, 9]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
GC_name = 'NGC3201'
data_path = os.getcwd()
fidanka = pd.read_csv('fiducial_lines.csv')

In [ ]:
df = pd.read_csv(GC_name + '_fitstars_DRCR.dat')

In [ ]:
from scipy.interpolate import CubicSpline
ff = CubicSpline(fidanka['v'],fidanka['vi'])

In [ ]:
AS_df = pd.read_csv("{}artstars.dat".format(GC_name),sep='\s+')
AS_df = AS_df[((AS_df['#InputX'] - AS_df['OutputX'])**2 + (AS_df['InputY'] - AS_df['OutputY'])**2 <= 0.25) & (np.abs(AS_df['InputF606W'] - AS_df['OutputF606W']) <= 0.75) & (np.abs(AS_df['InputF814W'] - AS_df['OutputF814W']) <= 0.75)]
V_diff=31.4
VI_diff=0.897
I_diff = V_diff - VI_diff
AS_df_CR = AS_df.copy()
AS_df_CR['InputF606W'] += V_diff
AS_df_CR['OutputF606W'] += V_diff
AS_df_CR['InputF814W'] += I_diff
AS_df_CR['OutputF814W'] += I_diff
AS_df_CR['AR_cdiff'] = (AS_df_CR['OutputF606W'] - AS_df_CR['OutputF814W']) - (AS_df_CR['InputF606W'] - AS_df_CR['InputF814W'])
df['Obs_cdiff'] = df['vi'] - ff(df['v'])

In [ ]:
plt.scatter(df['Obs_cdiff'],df['v'],s=0.1,alpha=0.5,c='r',label=r'Obs $\Delta_{V-I}$')
plt.scatter(AS_df_CR['AR_cdiff'], AS_df_CR['InputF606W'],s=0.1,alpha=0.5,c='y',label=r'AS $\Delta_{V-I}$')
plt.ylim(np.max(df['v']),np.min(df['v']))
plt.xlim(-0.1,0.1)
plt.legend(markerscale=10);

In [ ]:
AS_df_CR = AS_df_CR[np.abs(AS_df_CR['AR_cdiff']) < 0.08]

In [ ]:
from scipy.stats import median_abs_deviation
df['MAD'] = np.concatenate((np.zeros(50),np.array([median_abs_deviation(df['Obs_cdiff'].values[i:i+100]) for i in range(50)]),np.array([median_abs_deviation(df['Obs_cdiff'].values[i:i+200]) for i in range(len(df) - 200)]),np.zeros(100)))
AS_df_CR['MAD'] = np.concatenate((np.zeros(500),np.array([median_abs_deviation(AS_df_CR['AR_cdiff'].values[i:i+1000]) for i in range(len(AS_df_CR) - 1000)]),np.zeros(500)))

In [ ]:
for max_v in np.linspace(15.5,21,50):
    min_v = max_v - 1
    art_in_range = AS_df_CR[(AS_df_CR['InputF606W'] < max_v) & (AS_df_CR['InputF606W'] > min_v) & (AS_df_CR['AR_cdiff'] > -0.05) & (AS_df_CR['AR_cdiff'] < 0.05)]['AR_cdiff'].values
    obs_in_range = df[(df['v'] < max_v) & (df['v'] > min_v)]['Obs_cdiff'].values
    plt.scatter(max_v-0.5, median_abs_deviation(art_in_range),c='b')
    plt.scatter(max_v-0.5, median_abs_deviation(obs_in_range),c='r')

In [ ]:
mag_bin = []
ratio = []
for max_v in np.linspace(15.5,21,50):
    min_v = max_v - 1
    art_in_range = AS_df_CR[(AS_df_CR['InputF606W'] < max_v) & (AS_df_CR['InputF606W'] > min_v) & (AS_df_CR['AR_cdiff'] > -0.05) & (AS_df_CR['AR_cdiff'] < 0.05)]['AR_cdiff'].values
    obs_in_range = df[(df['v'] < max_v) & (df['v'] > min_v)]['Obs_cdiff'].values
    mag_bin.append(max_v-0.5)
    ratio.append(median_abs_deviation(art_in_range)/median_abs_deviation(obs_in_range))
    plt.scatter(max_v-0.5, median_abs_deviation(art_in_range)/median_abs_deviation(obs_in_range),c='b')

In [ ]:
mag_bin = []
diff = []
for max_v in np.linspace(15.5,21,50):
    min_v = max_v - 1
    art_in_range = AS_df_CR[(AS_df_CR['InputF606W'] < max_v) & (AS_df_CR['InputF606W'] > min_v) & (AS_df_CR['AR_cdiff'] > -0.05) & (AS_df_CR['AR_cdiff'] < 0.05)]['AR_cdiff'].values
    obs_in_range = df[(df['v'] < max_v) & (df['v'] > min_v)]['Obs_cdiff'].values
    mag_bin.append(max_v-0.5)
    diff.append(np.sqrt(np.abs(median_abs_deviation(art_in_range)**2 - median_abs_deviation(obs_in_range)**2)))
plt.scatter(mag_bin, diff,c='b')

In [ ]:
import piecewise_regression
pw_fit = piecewise_regression.Fit(mag_bin, diff, n_breakpoints=2)
pw_fit.plot_fit(color="red", linewidth=4)
plt.scatter(mag_bin,diff)

In [ ]:
pw_fit.summary()

In [ ]:
from scipy.interpolate import splrep, BSpline
spl = splrep(mag_bin,diff)
plt.scatter(np.linspace(15,20,100),BSpline(*spl)(np.linspace(15,20,100)))